In [ ]:
!pip install tune-sklearn ray ray[tune]

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
# from ray.tune.sklearn import TuneSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from ray import tune

np.random.seed(42)
data, y = torch.load('data/data_v2.pkl')
train_data, test_data = data.iloc[:len(y)], data.iloc[len(y):]
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )
std = StandardScaler()
std.fit(train_data)

model = LogisticRegressionCV(cv=10, penalty='elasticnet', solver='saga', l1_ratios=[.1, .5, .7, .9, .95, .99, 1])
model.fit(std.transform(train_data), y)

d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\project\python\.venv\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was re

LogisticRegressionCV(cv=10, l1_ratios=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
                     penalty='elasticnet', solver='saga')

In [2]:
model.score(std.transform(train_data), y)

0.8338945005611672

In [4]:
model.C_

array([0.04641589])

In [5]:
model.l1_ratio_

array([0.1])

In [7]:
from sklearn.linear_model import ElasticNet

net = ElasticNet(l1_ratio=.1, alpha=0.04641589)
net.fit(train_data, y)
net.score(train_data, y)


0.0

In [ ]:
cross_val_score(model, train_data, y, cv=10, n_jobs=-1)

In [3]:
import pandas as pd

# best_model = tune_search.best_estimator
best_model = model
y_test = best_model.predict(std.transform(test_data))
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('ElasticNet.csv')